In [ ]:
! sudo apt install nasm 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  nasm
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 359 kB of archives.
After this operation, 2,831 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 nasm amd64 2.13.02-0.1 [359 kB]
Fetched 359 kB in 0s (4,771 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfig


# Funciones en ASM

Conversión a arreglo de 8bits

La función mono_array_asm se encarga de convertir una imagen de 84x48 monocromática representada por ceros y unos a un arreglo de 504 elementos hexadecimal que pueda ser pasado al LCD.

In [ ]:
%%file mono_array_asm.asm
;mono_array_asm(unsigned char *img_mono, unsigned char *mono_array);

global mono_array_asm 
    section .text 

mono_array_asm:
;rdi <- *img_mono 
;rsi <- *mono_array

    mov r10,0  
    mov r11, 0  
    mov r8, 0
    mov r9, 0
    mov r13, 0
    mov r14, 0
    mov r15, 0

    mov r15, rdi ;img[0][0]
for_alto48:
    mov r11, 0
    mov rdi, r15 ;img[j+8][0]  
    add r15, 672 ;84*8

    mov r14, rdi ;copy latest img[i][0]
for_ancho84:
    mov rdi, r14 ;img[i][j=j+1] 
    inc r14
  
    mov r8, 0

    ;Binary to Decimal - 8bits vertically
    mov r13b, byte [rdi]
    mov r9, r13
    add r8, r9  ;0

    add rdi, 84
    mov r13b, byte [rdi]
    mov r9, r13
    shl r9, 1
    add r8, r9   ;1

    add rdi, 84
    mov r13b, byte [rdi]
    mov r9, r13
    shl r9, 2
    add r8, r9   ;2

    add rdi, 84
    mov r13b, byte [rdi]
    mov r9, r13
    shl r9, 3
    add r8, r9   ;3

    add rdi, 84
    mov r13b, byte [rdi]
    mov r9, r13
    shl r9, 4
    add r8, r9   ;4

    add rdi, 84
    mov r13b, byte [rdi]
    mov r9, r13
    shl r9, 5
    add r8, r9   ;5

    add rdi, 84
    mov r13b, byte [rdi]
    mov r9, r13
    shl r9, 6
    add r8, r9   ;6

    add rdi, 84
    mov r13b, byte [rdi]
    mov r9, r13
    shl r9, 7
    add r8, r9   ;7

    mov [rsi], r8

cont_ancho84:
    inc rsi ;Byte_index++

    inc r11
    cmp r11, 84
    jl for_ancho84

cont_alto48:
    add r10, 8
    cmp r10, 48
    jl for_alto48

done:
    ret


Writing mono_array_asm.asm


Filtro con umbralización de intensidad global

Comienza convirtiendo la imagen a escala de grises empleando pesos específicos para cada color para luego volverla monocromática empleando como umbral de división la media de intensidades.

In [ ]:
%%file mono_img_asm.asm
;mono_img_asm(unsigned char *img, unsigned char *img_ithresh, double weight_b, double weight_g, double weight_r)

global mono_img_asm 
    section .text 

mono_img_asm:
;rdi <- *img 
;rsi <- *output
;xmm0 <- weight_b
;xmm1 <- weight_g
;xmm2 <- weight_r

    xorpd xmm3,xmm3
    xorpd xmm4, xmm4
    xorpd xmm5,xmm5 
    xorpd xmm6,xmm6
    xorpd xmm7,xmm7
    mov r10, 0  
    mov r11, 0
    mov r13, 0

    mov r9, rdi ;salvando *img
    mov r12, rsi ;salvando *output

for_gray:
   ;Salvando constantes
    movsd xmm3, xmm0
    movsd xmm4, xmm1
    movsd xmm5, xmm2

    ;0.0722* img[index]
    mov r11b, byte [r9]
    cvtsi2sd xmm7, r11
    mulsd xmm3, xmm7
    add r9, 1   

    ;0.7152* img[index+1]
    mov r11b, byte [r9]    
    cvtsi2sd xmm7, r11      
    mulsd xmm4, xmm7
    add r9, 1     

    ;0.2126* img[index+2]
    mov r11b, byte [r9]
    cvtsi2sd xmm7, r11         
    mulsd xmm5, xmm7
    add r9, 1   

    ;promedio=suma de lo anterior
    xorpd xmm6, xmm6
    addsd xmm6, xmm3
    addsd xmm6, xmm4
    addsd xmm6, xmm5
    cvtsd2si r8, xmm6

continue_gray:
    mov [r12], r8
    inc r12
    add r13, r8     ;suma de todas las intensidades
    inc r10
    cmp r10, 4032   ;84x48
    jl for_gray

umbral:
    xorpd xmm3,xmm3 ;umbral
    xorpd xmm4, xmm4 
    mov r12, rsi    ;*recuperando *output[0]

    cvtsi2sd xmm3, r13
    cvtsi2sd xmm4, r10
    divsd xmm3, xmm4    ;umbral=suma de intensidades/cantidad de pixeles
    cvtsd2si r13, xmm3  ;r13=umbral

    mov r8, 0
    mov r10, 0
    mov r11, 0

for_umbral:
    mov r11b, byte [r12] 
    cmp r11, r13    ;imgn[i][j]>umbral
    jge white_pixel

black_pixel:
    mov r8, 1   

continue_umbral:
    mov byte [r12], r8b
    inc r12
    inc r10
    cmp r10, 4032   ;84x48
    jl for_umbral

done:
    ret

white_pixel:
    mov r8, 0
    jmp continue_umbral


Writing mono_img_asm.asm


Filtro sobel

Transforma la imagen recibida a escala de grises para luego detectar sus bordes y gradiente promedio empleando el filtro sobel. Finalmente, la vuelve monocromática empleando la media de gradientes como umbral.

In [ ]:
%%file sobel_asm.asm
;sobel_asm(unsigned char *img, unsigned char *sobel_img, unsigned char *sobel_arr, double *sobel_filtered_image,weight_b,weight_g,weight_r)

global sobel_asm 
    section .text 

sobel_asm:
;rdi <- *img 
;rsi <- *sobel_img
;rdx <- *sobel_arr
;rcx <- *sobel_filtered_image 
;xmm0 <- weight_b
;xmm1 <- weight_g
;xmm2 <- weight_r

    xorpd xmm3,xmm3
    xorpd xmm4, xmm4
    xorpd xmm5,xmm5 
    xorpd xmm6,xmm6
    xorpd xmm7,xmm7
    mov r10, 0  
    mov r11, 0
    mov r13, 0

    mov r9, rdi ;salvando *img[0][0][0]
    mov r12, rsi ;salvando *sobel_img[0][0]

for_gray:
   ;Salvando constantes
    movsd xmm3, xmm0
    movsd xmm4, xmm1
    movsd xmm5, xmm2

    ;0.0722* img[index]
    mov r11b, byte [r9]
    cvtsi2sd xmm7, r11
    mulsd xmm3, xmm7
    add r9, 1   
    ;0.7152* img[index+1]
    mov r11b, byte [r9]    
    cvtsi2sd xmm7, r11      
    mulsd xmm4, xmm7
    add r9, 1     
    ;0.2126* img[index+2]
    mov r11b, byte [r9]
    cvtsi2sd xmm7, r11         
    mulsd xmm5, xmm7
    add r9, 1   

    ;promedio=suma de lo anterior
    xorpd xmm6, xmm6
    addsd xmm6, xmm3
    addsd xmm6, xmm4
    addsd xmm6, xmm5
    cvtsd2si r8, xmm6

continue_gray:
    mov [r12], r8
    inc r12
    inc r10
    cmp r10, 4032   ;84x48
    jl for_gray

sobel:
    xorpd xmm3,xmm3 ;gx
    xorpd xmm4, xmm4 ;gy
    xorpd xmm5, xmm5 ;g
    xorpd xmm6, xmm6 
    xorpd xmm7, xmm7 
    xorpd xmm8, xmm8 ;umbral
    mov r8, 0
    mov r9, 0
    ;r10 vale 4032 para este punto
    mov r11, 0 
    mov r12, rsi    ;*recuperando *sobel_img[0][0]
    mov r14, 0
    mov r15, rcx 

    add r15, 672  
    ;comienza en i=1 (4bytes -> 84x8)

for_i:
    mov r8, 0
    mov r11, r12 

for_j:
    add r15, 8
    ;comienza en j=1 (4bytes)

    xorpd xmm3, xmm3    ;gx=0
    xorpd xmm4, xmm4    ;gy=0
    xorpd xmm5, xmm5    ;g = 0

    ;Convolución:
    mov r14b, byte [r12]
    cvtsi2sd xmm3, r14
    cvtsi2sd xmm4, r14
    ;gx+=1*img[i][j]
    ;gy+=1*img[i][j]
    inc r12 

    mov r14b, byte [r12]
    cvtsi2sd xmm6, r14
    addsd xmm4, xmm6
    addsd xmm4, xmm6
    ;gx+=0*img[i][j+1]
    ;gy+=2*img[i][j+1]
    inc r12 

    mov r14b, byte [r12]
    cvtsi2sd xmm6, r14
    subsd xmm3,xmm6
    addsd xmm4, xmm6
    ;gx+=-1*img[i][j+2]
    ;gy+=1*img[i][j+2]
    add r12, 82

    mov r14b, byte [r12]
    cvtsi2sd xmm6, r14
    addsd xmm3,xmm6
    addsd xmm3,xmm6
    ;gx+=2*img[i+1][j]
    ;gy+=0*img[i+1][j]
    inc r12

    ;gx+=0*img[i+1][j+1]
    ;gy+=0*img[i+1][j+1]
    inc r12 

    mov r14b, byte [r12]
    cvtsi2sd xmm6, r14
    subsd xmm3,xmm6
    subsd xmm3,xmm6
    ;gx+=-2*img[i+1][j+2]
    ;gy+=0*img[i+1][j+2]
    add r12, 82

    mov r14b, byte [r12]
    cvtsi2sd xmm6, r14
    addsd xmm3,xmm6
    subsd xmm4, xmm6
    ;gx+=1*img[i+2][j]
    ;gy+=-1*img[i+2][j]
    inc r12 

    mov r14b, byte [r12]
    cvtsi2sd xmm6, r14
    subsd xmm4, xmm6
    subsd xmm4, xmm6
    ;gx+=0*img[i+2][j+1]
    ;gy+=-2*img[i+2][j+1]
    inc r12 

    mov r14b, byte [r12]
    cvtsi2sd xmm6, r14
    subsd xmm3,xmm6 
    subsd xmm4, xmm6
    ;gx+=-1*img[i+2][j+2]
    ;gy+=-1*img[i+2][j+2] 

    mulsd xmm3, xmm3    ;gx^2
    mulsd xmm4, xmm4    ;gy^2
    addsd xmm3, xmm4    ;gx^2+gy^2
    sqrtsd xmm5, xmm3 
    ;g=sqrt(gx^2+gy^2)

cont_j:
    addsd xmm8, xmm5    ;umbral+=g
    movsd [r15], xmm5  ;sobel_filtered_img[i][j]=g
    inc r11
    mov r12, r11
    inc r8
    cmp r8, 82 
    jl for_j

cont_i:
    inc r12 
    inc r12
    add r15, 8
    add r15, 8

    inc r9
    cmp r9, 46
    jl for_i

umbral:
    sub r10, 260 ;bits totales=82x46, no 84x48
    cvtsi2sd xmm9, r10 
    divsd xmm8, xmm9    ;umbral=umbral/4032.0
    xorpd xmm3, xmm3

    mov r12, rsi    ;*recuperando *output[0]
    mov r15, rcx
    mov r8, 0
    mov r9, 0

for_umbral:
    movsd xmm3, [r15]
    ucomisd  xmm3, xmm8    ;imgn[i][j]<umbral
    jb white_pixel

black_pixel:
    mov r8, 1   

continue_umbral:
    mov byte [r12], r8b ;img_mono[i][j]
    inc r12
    movsd [r15], xmm3
    add r15, 8
    inc r9
    cmp r9, 4032   ;84x48
    jl for_umbral

done:
    ret

white_pixel:
    mov r8, 0
    jmp continue_umbral

Writing sobel_asm.asm


# Funciones en C

Enlaza los algoritmos en ensamblador con el resto de funciones en C. En C proporciona versiones propias de filtro grayscale, umbralización por media de intensidades, operador sobel, y conversor a array de LCD.

In [ ]:
%%file filters_c.c

#include <math.h>
#include <stdlib.h>
#include <stdio.h>

extern void mono_img_asm(unsigned char *img, unsigned char *img_ithresh, double weight_b, double weight_g, double weight_r);
extern void mono_array_asm(unsigned char *img_mono, unsigned char *mono_array);
extern void sobel_asm(unsigned char *img, unsigned char *sobel_img, unsigned char *sobel_arr, double *sobel_filtered_image, double weight_b, double weight_g, double weight_r);

float grayscale(unsigned char *img,unsigned char *img_grayscale){
    int promedio;
    int index, i , j;
    float umbral =0;
    for(i=0;i<48;i++){ //Alto
        for(j=0;j<84;j++){ //Ancho
            index=i*84*3+j*3;
            //Valor promedio pesado
            promedio=0.0722* img[index]+ 0.7152* img[index+1] + 0.2126* img[index+2];         
            index=i*84+j;
            //Asignando valor a img grayscale
            img_grayscale[index]=promedio;
            //printf("%d ", img_ithresh[index]);
            umbral+=promedio; //Calculando umbral para thresholding
        }
    }

    return (umbral/4032);    
}

//Conversión a formato de pantalla LCD
void To_array(unsigned char *img_mono, unsigned char *img_arr){
    int Byte_index=0;
    for(int i=0;i<48;i=8+i){
        for(int j=0;j<84;j++){
            for(int n=0; n<8; n++){
                img_arr[Byte_index]+=(img_mono[(i+n)*84+j]*pow(2, n));
            }
            Byte_index++;
        }
    }
}

void Int_thresh_c(unsigned char *img, unsigned char *img_ithresh, unsigned char *img_arr)
{
    //Convertir a grayscale 
    float umbral;
    umbral = grayscale(img, img_ithresh);
    int index,i,j;

    //Transformando a monocromática: 
    //  Index>umbral -> Píxel blanco
    //  Index<umbral -> Píxel negro
    unsigned char img_mono[48][84];
    for(i=0;i<48;i++){
        for(j=0;j<84;j++){
            index=i*84+j;
            if(img_ithresh[index]>umbral){
                img_ithresh[index]=255;
                img_mono[i][j]=0;
            }else{
                img_ithresh[index]=0;
                img_mono[i][j]=1;
            }
        }
    }

    To_array(img_mono[0], img_arr);

    return;
}

void Sobel_c(unsigned char *img, unsigned char *sobel_img, unsigned char *sobel_arr, float *sobel_filtered_image)
{
    float umbral = grayscale(img,sobel_img);
    float sobelx[9] = {1.0, 0.0, -1.0, 2.0, 0.0, -2.0, 1.0, 0.0, -1.0};
    float sobely[9] = {1.0, 2.0, 1.0, 0.0, 0.0, 0.0, -1.0, -2.0, -1.0};
    float media = 0.0;
    int i,j,m,n,index;
    float gx,gy,g;
    for (i=1;i<47;i++){
        for (j=1;j<83;j++){
            index=0;
            gx=0.0;
            gy=0.0;
            for (m=i-1;m<i+2;m++){ //Convolución por el kernel
                for (n=j-1;n<j+2;n++){
                    gx+=sobelx[index]*sobel_img[m*84+n];
                    gy+=sobely[index]*sobel_img[m*84+n];
                    index++;
                }
            }
            g = sqrt(gx*gx+gy*gy);
            sobel_filtered_image[i*84+j]=g; //index-> [(i+1)*84+j+1]
            media+=g;
        }
    }
    media=media/(82*46);

    //Transformando a monocromática: 
    //  Index<umbral -> Píxel blanco
    //  Index>umbral -> Píxel negro
    unsigned char img_mono[48][84];
    for(i=0;i<48;i++){
        for(j=0;j<84;j++){
            if(sobel_filtered_image[i*84+j]<umbral){
                sobel_img[i*84+j]=255;
                img_mono[i][j]=0;
            }else{
                sobel_img[i*84+j]=0;
                img_mono[i][j]=1;
            }
        }
    }

    //Transformando a arreglo en bytes compatible con pantalla LCD
    To_array(img_mono[0],sobel_arr);

    return;
}


Writing filters_c.c


In [ ]:
! nasm -f elf64 mono_img_asm.asm -o mono_img_asm.o 
! nasm -f elf64 sobel_asm.asm -o sobel_asm.o 
! nasm -f elf64 mono_array_asm.asm -o mono_array_asm.o 
! gcc -c -fpic filters_c.c -o filters_c.o
! gcc -shared filters_c.o mono_img_asm.o mono_array_asm.o sobel_asm.o  -o filters_c.so



#**Funciones en Python** 

Importando librerías.

In [ ]:
import cv2 as cv
import numpy as np
import ctypes
import math 

Funciones de conversión a array de LCD, filtro sobel, umbralización por media de intensidades y grayscale en python. Adicionalmente, incluye la función de exportar array para escribir los archivos de texto de los arreglos para LCD en hexadecimal.

In [ ]:
#CONVERSION A ARRAY DE PANTALLA NOKIA LCD
def NokiaLCD_array(img_mono, img_arr):
    Byte_index = 0
    for i in range(0, 48, 8): #Alto
        for j in range(0, 84): #Ancho
            #Transformando a bloques de 8bits - forma: valor*2^(n)
            for n in range(8):
                img_arr[Byte_index]=img_arr[Byte_index]+img_mono[i+n][j]*pow(2,n)
            #Cuando se cumple 8bits, se pasa al siguiente bloque
            Byte_index= Byte_index+1    
    return img_arr

#FILTRO SOBEL
def Sobel_py(img,img_mono,img_arr,sobel_filtered_image):
    sobelx = [1.0, 0.0, -1.0, 2.0, 0.0, -2.0, 1.0, 0.0, -1.0]
    sobely = [1.0, 2.0, 1.0, 0.0, 0.0, 0.0, -1.0, -2.0, -1.0]
    media = 0.0

    for i in range(1, 47): #Alto
        for j in range(1, 83): #Ancho
            #Calculate gx and gy using Sobel (horizontal and vertical gradients)
            index, gx, gy = 0, 0.0, 0.0
            for m in range(i-1, i+2):
                for n in range(j-1, j+2):
                    gx+=sobelx[index]*img[m][n]
                    gy+=sobely[index]*img[m][n]
                    index+=1

            g= math.sqrt(gx ** 2 + gy ** 2)
            sobel_filtered_image[i][j] = g
            media = media+g
    
    media = media/(82*46)

    #Thresholding de la imagen a blanco o negro
    for i in range(0, 48): #Alto
        for j in range(0, 84): #Ancho
            if sobel_filtered_image[i][j] < media:    #Si el pixel es menor al umbral, se guarda 0 en el arreglo dado que es de fondo (pixel no prendido en pantalla)
                img_mono[i][j]=0
                img[i][j]=255
            else:   
                img[i][j]=0
    
    #Array para NokiaLCD
    img_arr=NokiaLCD_array(img_mono, img_arr)

    return img, img_arr

#FILTRO UMBRAL INTENSIDAD GLOBAL
def Int_thresh_py(img,img_mono,img_arr):
    umbral=0.0
    cont=0
    for i in range(0, 48): #Alto
        for j in range(0, 84): #Ancho        
            if(img[i][j]!=255):
                umbral=umbral+img[i][j]
                cont=cont+1
    umbral=umbral/cont
    #print("Umbral: ", umbral, "Cont:", cont)

    #Thresholding de la imagen a blanco o negro
    for i in range(0, 48): #Alto
        for j in range(0, 84): #Ancho
            if img[i][j] > umbral: #Global Threshold
                img[i][j]=255
                img_mono[i][j]=0
            else:
                img[i][j]=0
    
    #Array para NokiaLCD
    img_arr=NokiaLCD_array(img_mono, img_arr)    
    
    return img, img_arr

#GRAYSCALE FILTER 
def grayscale(img):
    gray = np.zeros((48, 84),dtype='uint8')
    for i in range (len(img)):
        for j in range(len(img[0])):
            # Gray = 0.0722Blue + 0.7152Green + 0.2126Red
            gray[i][j]=int(0.0722*img[i][j][0]+0.7152*img[i][j][1]+0.2126*img[i][j][2])
    return gray

def exportar_arr(img_arr,file_name):
    #Escribiendo el arreglo en un archivo de texto
    file = open(file_name, "w+")
    file.write("char bitmapArray[504] = {")
    for i in range(len(img_arr)-1):
        content = hex(img_arr[i])
        file.write(content)
        file.write(", ")
        if (i % 80)==0:
            file.write("\n")
    content = hex(img_arr[len(img_arr)-1])
    file.write(content)
    file.write("};")
    file.close()

Función de interpolación bilineal en python para reducir la imagen a una resolución de 84x48 píxeles. Se adaptó de código referencial incluido en: https://meghal-darji.medium.com/implementing-bilinear-interpolation-for-image-resizing-357cbb2c2722

In [ ]:
def resizing_interpolacion_bilineal(imagen_original, nuevo_altura, nuevo_ancho):
    #Dimensiones originales de la imagen
    altura_original = len(imagen_original)
    ancho_original = len(imagen_original[0])
    canales_imagen = 3 #RGB

    nueva_imagen = np.zeros((nuevo_altura, nuevo_ancho, canales_imagen))

    #Factores de escalamiento para la altura y el ancho
    if nuevo_ancho != 0:
        factor_escala_ancho = ancho_original/nuevo_ancho
    else:
        factor_escala_ancho = 0
    
    if nuevo_altura != 0:
        factor_escala_altura = altura_original/nuevo_altura
    else:
        factor_escala_altura = 0

    #Se recorren todos los pixeles de la nueva imagen mediante dos iterativas
    for i in range(nuevo_altura):
        for j in range(nuevo_ancho):
            #Coordenadas equivalentes de la img original
            x = factor_escala_altura*i
            y = factor_escala_ancho*j
            #Calculando los 4 píxeles de la imagen original más cercanos a x e y 
            x_inferior= int(x)
            if (altura_original-1) <= int(x+1):
                x_superior = altura_original-1
            else:
                x_superior = int(x+1)

            y_inferior = int(y)
            if (ancho_original-1) <= int(y+1):
                y_superior = ancho_original -1
            else:
                y_superior =  int(y+1)
            
            #Si los valores de x e y son valores enteros, no hay necesidad de interpolar, sino
            #asignar los valores de la imagen original a dichas coordenadas 
            if(x_superior == x_inferior) and (y_superior == y_inferior):
                valor_interpolado = imagen_original[int(x), int(y), :]
            #Si x es entero, se interpola la parte solo y
            elif(x_superior == x_inferior):
                valor_vecino1 = imagen_original[int(x), int(y_inferior), :]
                valor_vecino2 = imagen_original[int(x), int(y_superior), :]
                valor_interpolado = valor_vecino1*(y_superior-y)+valor_vecino2*(y-y_inferior)
            #Si y es entero, se interpola la parte solo x
            elif(y_superior == y_inferior):
                valor_vecino1 = imagen_original[int(x_inferior), int(y), :]
                valor_vecino2 = imagen_original[int(x_superior), int(y), :]
                valor_interpolado = (valor_vecino1)*(x_superior-x) + (valor_vecino2)*(x-x_inferior)
            #Si x e y no son enteros, se interpolan ambos valores
            else:
                #Calculo de los valores de los 4 vecinos que se encuentran cercanos a x e y
                valor_vecino1 = imagen_original[x_inferior, y_inferior, :]
                valor_vecino2 = imagen_original[x_superior, y_inferior, :]
                valor_vecino3 = imagen_original[x_inferior, y_superior, :]
                valor_vecino4 = imagen_original[x_superior, y_superior, :]
                #Cálculo de la contribución en un valor interpolado 1 y 2 de los vecinos 1 y 2, así como 3 y 4 respectivamente
                valor_interpolado1 = valor_vecino1*(x_superior-x)+valor_vecino2*(x-x_inferior)
                valor_interpolado2 = valor_vecino3*(x_superior-x)+valor_vecino4*(x-x_inferior)
                #Cálculo de la contribución de los valores interpolados 1 y 2 en un valor interpolado total que ocupara las coordenas i y j
                #de la nueva imagen
                valor_interpolado = (valor_interpolado1)*(y_superior-y)+(valor_interpolado2)*(y-y_inferior)

            nueva_imagen[i,j,:] = valor_interpolado
    return nueva_imagen.astype(np.uint8)

# FUNCIÓN PRINCIPAL

Se enlazan todas las funciones mediante ctypes y selecciona una imagen a convertir. Empleando todas las funciones, se guardan versiones de la imagen convertida por ambos métodos en los tres lenguajes (python/C/asm).
Para poder usar el programa, se deben tener creadas las carpetas "Results" y "Arrays".

In [ ]:
if __name__ == '__main__':

#---------------------------------------------------------------------------------------------------#      
#-----------------------------------------CTYPES FUNCTIONS------------------------------------------#      
#---------------------------------------------------------------------------------------------------#    
    
    lib = ctypes.CDLL('./filters_c.so')
    lib.Sobel_c.argtypes = [np.ctypeslib.ndpointer(dtype=np.ubyte, ndim=3), np.ctypeslib.ndpointer(dtype=np.ubyte, ndim=2), 
                            np.ctypeslib.ndpointer(dtype=np.ubyte), np.ctypeslib.ndpointer(dtype=float)]
    lib.Int_thresh_c.argtypes = [np.ctypeslib.ndpointer(dtype=np.ubyte, ndim=3), np.ctypeslib.ndpointer(dtype=np.ubyte, ndim=2),  
                                np.ctypeslib.ndpointer(dtype=np.ubyte)]
    lib.mono_img_asm.argtypes = [np.ctypeslib.ndpointer(dtype=np.ubyte, ndim=3), np.ctypeslib.ndpointer(dtype=np.ubyte, ndim=2), 
                                ctypes.c_double, ctypes.c_double, ctypes.c_double]
    lib.sobel_asm.argtypes = [np.ctypeslib.ndpointer(dtype=np.ubyte, ndim=3), np.ctypeslib.ndpointer(dtype=np.ubyte, ndim=2), 
                            np.ctypeslib.ndpointer(dtype=np.ubyte), np.ctypeslib.ndpointer(dtype=float), ctypes.c_double, ctypes.c_double, ctypes.c_double]
    lib.mono_array_asm.argtypes = [np.ctypeslib.ndpointer(dtype=np.ubyte, ndim=2), np.ctypeslib.ndpointer(dtype=np.ubyte)]

    #Reading and resizing selected image
    img = cv.imread('Pikachu.png') 
    img = resizing_interpolacion_bilineal(img, 48, 84)
    #img = cv.resize(img,(84,48), interpolation=cv.INTER_CUBIC)
    cv.imwrite('Results/Resized.png',img)  

#---------------------------------------------------------------------------------------------------#      
#------------------------------------------VARIABLES------------------------------------------------#      
#---------------------------------------------------------------------------------------------------#    

    #VARIABLES: SOBEL
    #ASM
    Sobel_arrASM = np.zeros((505),dtype='uint8')   #Imagen en arreglo hexadecimal
    Sobel_monoASM = np.ones((48, 84),dtype='uint8') #Imagen en forma 0 y 1   
    sobel_filtered_imageASM = np.zeros((48, 84),dtype='float64')  
    #C
    Sobel_arrC = np.zeros((504),dtype='uint8')   #Imagen en arreglo hexadecimal
    Sobel_monoC = np.ones((48, 84),dtype='uint8') #Imagen en forma 0 y 1   
    sobel_filtered_imageC = np.zeros((48, 84),dtype='float64')  
    #Python
    Sobel_arrPy = np.zeros((504),dtype='uint8')   #Imagen en arreglo hexadecimal
    Sobel_monoPy = np.ones((48, 84),dtype='uint8') #Imagen en forma 0 y 1 
    sobel_filtered_imagePy = np.zeros((48, 84),dtype='f') 

    #VARIABLES: INTENSITY THRESHOLD 
    #ASM
    Ithresh_arrASM = np.zeros((505),dtype='uint8')   #Imagen en arreglo hexadecimal
    Ithresh_monoASM = np.zeros((48, 84),dtype='uint8') #Imagen en forma 0 y 1
    #C
    Ithresh_arrC = np.zeros((504),dtype='uint8')   #Imagen en arreglo hexadecimal
    Ithresh_monoC = np.ones((48, 84),dtype='uint8') #Imagen en forma 0 y 1  
    #Py
    Ithresh_arrPy = np.zeros((504),dtype='uint8')   #Imagen en arreglo hexadecimal
    Ithresh_monoPy = np.ones((48, 84),dtype='uint8') #Imagen en forma 0 y 1  

#---------------------------------------------------------------------------------------------------#    
#--------------------------------------SOBEL FILTER-------------------------------------------------#        
#---------------------------------------------------------------------------------------------------#        

    #Sobel ASM
    lib.sobel_asm(img, Sobel_monoASM,Sobel_arrASM,sobel_filtered_imageASM, 0.0722, 0.7152, 0.2126)  
    lib.mono_array_asm(Sobel_monoASM,Sobel_arrASM)
    cv.imwrite('Results/Sobel_ASM.png',Sobel_monoASM)
    Sobel_arrASM=np.delete(Sobel_arrASM, 504, 0)
    exportar_arr(Sobel_arrASM, "Arrays/Sobel_ASM.txt")
    # Para ver la imgn en blanco y negro cambiar  sobel_asm:
    #   -En white_pixel -> mov r8, 255
    #   -En black_pixel -> mov r8,0

    #Sobel C
    lib.Sobel_c(img, Sobel_monoC,Sobel_arrC,sobel_filtered_imageC)
    cv.imwrite('Results/Sobel_c.png',Sobel_monoC)
    exportar_arr(Sobel_arrC, "Arrays/Sobel_c.txt")

    #Sobel Python
    img_sobelPy = grayscale(img)
    img_sobel, Sobel_arr = Sobel_py(img_sobelPy,Sobel_monoPy,Sobel_arrPy,sobel_filtered_imagePy)
    cv.imwrite('Results/Sobel_py.png',img_sobelPy)    
    exportar_arr(Sobel_arr, "Arrays/Sobel_py.txt")

#---------------------------------------------------------------------------------------------------#      
#--------------------------------INTENSITY THRESHOLD------------------------------------------------#      
#---------------------------------------------------------------------------------------------------#      

    #GLOBAL INTENSITY THRESHOLD ASM  
    lib.mono_img_asm(img, Ithresh_monoASM, 0.0722, 0.7152, 0.2126)  
    lib.mono_array_asm(Ithresh_monoASM,Ithresh_arrASM)
    cv.imwrite('Results/Intensity_ASM.png',Ithresh_monoASM) 
    Ithresh_arrASM=np.delete(Ithresh_arrASM, 503, 0)
    exportar_arr(Ithresh_arrASM, "Arrays/Intensity_ASM.txt")
    # Para ver la imgn en blanco y negro cambiar  mono_img_asm:
    #   -En white_pixel -> mov r8, 255
    #   -En black_pixel -> mov r8,0

    #GLOBAL INTENSITY THRESHOLD C   
    lib.Int_thresh_c(img, Ithresh_monoC,Ithresh_arrC)
    cv.imwrite('Results/Intensity_C.png',Ithresh_monoC)
    exportar_arr(Ithresh_arrC, "Arrays/Intensity_C.txt")  

    #GLOBAL INTENSITY THRESHOLD FILTER
    img_ithreshPy = grayscale(img)
    img_ithreshPy, Ithresh_arrPy = Int_thresh_py(img_ithreshPy,Ithresh_monoPy,Ithresh_arrPy)
    exportar_arr(Ithresh_arrPy, "Arrays/Intensity_py.txt")
    cv.imwrite('Results/Intensity_py.png',img_ithreshPy)
